In [1]:
#importing all necessary libraries
import pandas as pd
import numpy as np
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
import itertools
import operator
from sklearn import metrics
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC, SMOTE, RandomOverSampler #using these oversampling methods, since the original data set is imbalanced
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Using TensorFlow backend.


In [1]:
#loading data (file 'example_data.csv' with a separator ';')
#columns in this example: 'sensor role', 'supply chain leg', 'distance to previous generated event (leg_rel)',
#'distance to expected next event (leg_abs)', 'atmosphere temperature at current location (temp_p)', 'setpoint deviation (spd)',
#'slope of two recent measurements', 'average deviation before a triggered alarm within one hour',
#'average deviation after a triggered alarm within one next hour' (estimated with the help of random forest regressor),
#'lower threshold', 'higher threshold', 'alarm label' (target feature)
df = pd.read_csv('example_data.csv', sep = ';')

In [3]:
#rescaling ambient temperature in terms of setpoint deviation units
#df.iloc[i, 10] corresponds to higher threshold, df.iloc[i, 9] - to lower, and df.iloc[i, 4] contains the initial unscaled ambient temperature value
for i in range(len(df)):
    middle = (df.iloc[i, 10] - df.iloc[i, 9])/2 + df.iloc[i, 9]
    span = (df.iloc[i, 10] - df.iloc[i, 9])/2
    if df.iloc[i, 4] > middle:
        df.iloc[i, 4] = (df.iloc[i, 4] - middle)/span
    elif df.iloc[i, 4] < middle:
        df.iloc[i, 4] = (df.iloc[i, 4] - middle)/span
    else:
        df.iloc[i, 4] = 0

In [4]:
#using only needed columns for predictor features and a target feature
y = df['label']
X = df[['sen_role', 'sc_leg', 'leg_rel', 'leg_abs', 'temp_p', 'spd', 'slope', 'db_1h', 'da_1h']]

In [5]:
#converting categorical variables into dummy variables and creating additional columns for this purpose
X = pd.get_dummies(X)

In [6]:
#deleting columns that do not contain additional information
X = X.drop(['sen_role_AMB', 'sc_leg_e'], axis = 1)

In [2]:
#checking whether conversion went without errors
X.head()

In [8]:
#specifying predictor features from which feature (sub)sets will be built and compared
features = [['leg_rel'], ['leg_abs'], ['temp_p'], ['spd'], ['slope'], ['db_1h'], ['da_1h'], ['sen_role_REG'],
            ['sc_leg_f', 'sc_leg_h', 'sc_leg_p']] #supply chain leg is represented by three dummy features that cannot be separated from each other, therefore they are lited as one element in a list

In [9]:
loops = loops #instead of 'loops' specify the value for a number of evaluations based on which comparison of scores will undertaken
#(the higher the value, the more reliable the comparison; minimum recommended value: 100)

Feature (sub)set with seven strongest features

In [3]:
comb = list(itertools.combinations(features, 7)) #iterator for feature combinations

#creating dictionaries for different evaluation metrics that will collect feature (sub)sets and their scores
#evaluation metrics include accuracy (ACC), area under curve (AUC), Gini coefficient (GIN), precision (PR), recall (REC) and their standard deviations
ACC7 = {}
AUC7 = {}
GIN7 = {}
PR7 = {}
REC7 = {}
ACCstd7 = {}
AUCstd7 = {}
GINstd7 = {}
PRstd7 = {}
RECstd7 = {}

#specifying what features are categorical (for a correct oversampling strategy later)
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']

learner = ExampleClassifier(optimal_parameters) #this line should be replaced by the initialization of any of the classifiers compared and optimal parameters are specified based on grid hyperparameter selection procedure

#loops for each feature (sub)set (irrespective of the order of features in a (sub)set)
for i in range(len(comb)):
    acc_7 = []
    auc_7 = []
    pr_7 = []
    rec_7 = []
    gin_7 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    
    #repeating evaluations 'loops' number of times for a comparison later
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_7.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_7.append(m)
            gin_7.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_7.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_7.append(m)
    
    #appening each of the dictionaries (key: name of a feature (sub)set, value: average evaluation metric value)
    ACC7[str(comb[i])] = sum(acc_7)/len(acc_7)
    ACCstd7[str(comb[i])] = np.std(acc_7)
    AUC7[str(comb[i])] = sum(auc_7)/len(auc_7)
    AUCstd7[str(comb[i])] = np.std(auc_7)
    GIN7[str(comb[i])] = sum(gin_7)/len(gin_7)
    GINstd7[str(comb[i])] = np.std(gin_7)
    PR7[str(comb[i])] = sum(pr_7)/len(pr_7)
    PRstd7[str(comb[i])] = np.std(pr_7)
    REC7[str(comb[i])] = sum(rec_7)/len(rec_7)
    RECstd7[str(comb[i])] = np.std(rec_7)

In [4]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)

Feature (sub)set with six strongest features

In [5]:
#the procedure is analogical to the search of a feature (sub)set containing seven features, so different parts of code will not be commented
comb = list(itertools.combinations(features, 6))
ACC6 = {}
AUC6 = {}
GIN6 = {}
PR6 = {}
REC6 = {}
ACCstd6 = {}
AUCstd6 = {}
GINstd6 = {}
PRstd6 = {}
RECstd6 = {}
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']
learner = ExampleClassifier(optimal_parameters)
for i in range(len(comb)):
    acc_6 = []
    auc_6 = []
    pr_6 = []
    rec_6 = []
    gin_6 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_6.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_6.append(m)
            gin_6.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_6.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_6.append(m)
    ACC6[str(comb[i])] = sum(acc_6)/len(acc_6)
    ACCstd6[str(comb[i])] = np.std(acc_6)
    AUC6[str(comb[i])] = sum(auc_6)/len(auc_6)
    AUCstd6[str(comb[i])] = np.std(auc_6)
    GIN6[str(comb[i])] = sum(gin_6)/len(gin_6)
    GINstd6[str(comb[i])] = np.std(gin_6)
    PR6[str(comb[i])] = sum(pr_6)/len(pr_6)
    PRstd6[str(comb[i])] = np.std(pr_7)
    REC6[str(comb[i])] = sum(rec_6)/len(rec_6)
    RECstd6[str(comb[i])] = np.std(rec_6)

In [6]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)

Feature (sub)set with five strongest features

In [7]:
#the procedure is analogical to the search of a feature (sub)set containing seven features, so different parts of code will not be commented
comb = list(itertools.combinations(features, 5))
ACC5 = {}
AUC5 = {}
GIN5 = {}
PR5 = {}
REC5 = {}
ACCstd5 = {}
AUCstd5 = {}
GINstd5 = {}
PRstd5 = {}
RECstd5 = {}
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']
learner = ExampleClassifier(optimal_parameters)
for i in range(len(comb)):
    acc_5 = []
    auc_5 = []
    pr_5 = []
    rec_5 = []
    gin_5 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_5.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_5.append(m)
            gin_5.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_5.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_5.append(m)
    ACC5[str(comb[i])] = sum(acc_5)/len(acc_5)
    ACCstd5[str(comb[i])] = np.std(acc_5)
    AUC5[str(comb[i])] = sum(auc_5)/len(auc_5)
    AUCstd5[str(comb[i])] = np.std(auc_5)
    GIN5[str(comb[i])] = sum(gin_5)/len(gin_5)
    GINstd5[str(comb[i])] = np.std(gin_5)
    PR5[str(comb[i])] = sum(pr_5)/len(pr_5)
    PRstd5[str(comb[i])] = np.std(pr_5)
    REC5[str(comb[i])] = sum(rec_5)/len(rec_5)
    RECstd5[str(comb[i])] = np.std(rec_5)

In [8]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)

Feature (sub)set with four strongest features

In [12]:
#the procedure is analogical to the search of a feature (sub)set containing seven features, so different parts of code will not be commented
comb = list(itertools.combinations(features, 4))
ACC4 = {}
AUC4 = {}
GIN4 = {}
PR4 = {}
REC4 = {}
ACCstd4 = {}
AUCstd4 = {}
GINstd4 = {}
PRstd4 = {}
RECstd4 = {}
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']
learner = ExampleClassifier(optimal_parameters)
for i in range(len(comb)):
    acc_4 = []
    auc_4 = []
    pr_4 = []
    rec_4 = []
    gin_4 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_4.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_4.append(m)
            gin_4.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_4.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_4.append(m)
    ACC4[str(comb[i])] = sum(acc_4)/len(acc_4)
    ACCstd4[str(comb[i])] = np.std(acc_4)
    AUC4[str(comb[i])] = sum(auc_4)/len(auc_4)
    AUCstd4[str(comb[i])] = np.std(auc_4)
    GIN4[str(comb[i])] = sum(gin_4)/len(gin_4)
    GINstd4[str(comb[i])] = np.std(gin_4)
    PR4[str(comb[i])] = sum(pr_4)/len(pr_4)
    PRstd4[str(comb[i])] = np.std(pr_4)
    REC4[str(comb[i])] = sum(rec_4)/len(rec_4)
    RECstd4[str(comb[i])] = np.std(rec_4)

In [9]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)

Feature (sub)set with three strongest features

In [13]:
#the procedure is analogical to the search of a feature (sub)set containing seven features, so different parts of code will not be commented
comb = list(itertools.combinations(features, 3))
ACC3 = {}
AUC3 = {}
GIN3 = {}
PR3 = {}
REC3 = {}
ACCstd3 = {}
AUCstd3 = {}
GINstd3 = {}
PRstd3 = {}
RECstd3 = {}
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']
learner = ExampleClassifier(optimal_parameters)
for i in range(len(comb)):
    acc_3 = []
    auc_3 = []
    pr_3 = []
    rec_3 = []
    gin_3 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_3.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_3.append(m)
            gin_3.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_3.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_3.append(m)
    ACC3[str(comb[i])] = sum(acc_3)/len(acc_3)
    ACCstd3[str(comb[i])] = np.std(acc_3)
    AUC3[str(comb[i])] = sum(auc_3)/len(auc_3)
    AUCstd3[str(comb[i])] = np.std(auc_3)
    GIN3[str(comb[i])] = sum(gin_3)/len(gin_3)
    GINstd3[str(comb[i])] = np.std(gin_3)
    PR3[str(comb[i])] = sum(pr_3)/len(pr_3)
    PRstd3[str(comb[i])] = np.std(pr_3)
    REC3[str(comb[i])] = sum(rec_3)/len(rec_3)
    RECstd3[str(comb[i])] = np.std(rec_3)

In [10]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)

Feature (sub)set with two strongest features

In [14]:
#the procedure is analogical to the search of a feature (sub)set containing seven features, so different parts of code will not be commented
comb = list(itertools.combinations(features, 2))
ACC2 = {}
AUC2 = {}
GIN2 = {}
PR2 = {}
REC2 = {}
ACCstd2 = {}
AUCstd2 = {}
GINstd2 = {}
PRstd2 = {}
RECstd2 = {}
categorical = ['sen_role_REG', 'sc_leg_f', 'sc_leg_h', 'sc_leg_p']
learner = ExampleClassifier(optimal_parameters)
for i in range(len(comb)):
    acc_2 = []
    auc_2 = []
    pr_2 = []
    rec_2 = []
    gin_2 = []
    cols = []
    for j in range(len(comb[i])):
        for o in range(len(comb[i][j])):
            cols.append(comb[i][j][o])
    X_new = X[cols]
    col_list = list(X_new.columns)
    cat_feat = []
    for k in categorical:
        try:
            cat_feat.append(col_list.index(k))
        except:
            pass
    if len(cat_feat) < len(col_list) and len(cat_feat) != 0:
        smtnc = SMOTENC(categorical_features = cat_feat, sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    elif len(cat_feat) == len(col_list):
        smtnc = RandomOverSampler(sampling_strategy = 0.5, random_state = 0)
    elif len(cat_feat) == 0:
        smtnc = SMOTE(sampling_strategy = 0.5, random_state = 0, n_jobs = -1)
    else:
        print('Error')
    model = Pipeline([('smtnc', smtnc), ('clf', learner)])
    for l in range(loops):
        acc = cross_val_score(model, X_new, y, scoring = 'accuracy', cv = 10)
        for m in acc:
            acc_2.append(m)
        auc = cross_val_score(model, X_new, y, scoring = 'roc_auc', cv = 10)
        for m in auc:
            auc_2.append(m)
            gin_2.append(2*m - 1)
        pr = cross_val_score(model, X_new, y, scoring = 'precision', cv = 10)
        for m in pr:
            pr_2.append(m)
        rec = cross_val_score(model, X_new, y, scoring = 'recall', cv = 10)
        for m in rec:
            rec_2.append(m)
    ACC2[str(comb[i])] = sum(acc_2)/len(acc_2)
    ACCstd2[str(comb[i])] = np.std(acc_2)
    AUC2[str(comb[i])] = sum(auc_2)/len(auc_2)
    AUCstd2[str(comb[i])] = np.std(auc_2)
    GIN2[str(comb[i])] = sum(gin_2)/len(gin_2)
    GINstd2[str(comb[i])] = np.std(gin_2)
    PR2[str(comb[i])] = sum(pr_2)/len(pr_2)
    PRstd2[str(comb[i])] = np.std(pr_2)
    REC2[str(comb[i])] = sum(rec_2)/len(rec_2)
    RECstd2[str(comb[i])] = np.std(rec_2)

In [11]:
#after evaluation has been completed, user can take a look into each dictionary and make a decision, given multiple evaluation metrics, what feature (sub)set(s) perfromed best
#apart from competing cases, in which all (or most) evaluation metrics from one feature (sub)set are not consistently stronger than from another, the following tests should be performed for separate evaluation metrics
#non-parametric Kruskal-Wallis test (scipy.stats.mstats.kruskalwallis or scipy.stats.kruskal) and non-parametric pairwise Mann-Whitney U test (scipy.stats.mannwhitneyu)